In [1]:
# Jovian Commit Essentials
# Please retain and execute this cell without modifying the contents for `jovian.commit` to work
!pip install jovian --upgrade -q
import jovian
jovian.set_project('02-linear-regression')
jovian.set_colab_id('1x0FHRUdMFYeBm9WcZCV2Gx7ZJwJPB0S6')

     |████████████████████████████████| 68 kB 2.7 MB/s 


In [2]:
# Uncomment and run the appropriate command for your operating system, if required

# Linux / Binder
# !pip install numpy torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# Windows 
!pip install numpy torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio==0.7.0 -f https://download.pytorch.org/whl/torch_stable.html

# MacOS
# !pip install numpy torch torchvision torchaudio

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 159.3 MB 3.6 kB/s 
     |████████████████████████████████| 11.8 MB 2.5 MB/s 
     |████████████████████████████████| 7.6 MB 5.3 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.11.1+cu111
    Uninstalling torchvision-0.11.1+cu111:
      Successfully uninstalled torchvision-0.11.1+cu111
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.10.0+cu111
    Uninstalling torchaudio-0.10.0+cu111:
      Successfully uninstalled torchaudio-0.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.11.0 requires torch

In [3]:
import numpy as np
import torch

In [4]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [5]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [6]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


We've separated the input and target variables because we'll operate on them separately. Also, we've created numpy arrays, because this is typically how you would work with training data: read some CSV files as numpy arrays, do some processing, and then convert them to PyTorch tensors.

Let's convert the arrays to PyTorch tensors.

In [7]:
# Weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[ 1.0989,  1.5503,  0.6631],
        [-0.8992,  0.5920,  0.6517]], requires_grad=True)
tensor([ 0.6968, -0.7305], requires_grad=True)


`torch.randn` creates a tensor with the given shape, with elements picked randomly from a [normal distribution](https://en.wikipedia.org/wiki/Normal_distribution) with mean 0 and standard deviation 1.

Our *model* is simply a function that performs a matrix multiplication of the `inputs` and the weights `w` (transposed) and adds the bias `b` (replicated for each observation).

![matrix-mult](https://i.imgur.com/WGXLFvA.png)

We can define the model as follows:

In [8]:
#Here we are defining the model
def model(x):
  return x @ w.t()+b

In [9]:
#Generate prediction
preds=model(inputs)
print(preds)

tensor([[213.2935,   1.3104],
        [279.5530,  11.2412],
        [342.4912,  38.1599],
        [203.9757, -42.8857],
        [271.7586,  39.6706]], grad_fn=<AddBackward0>)


In [10]:
#Compare with targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


## Loss function

Before we improve our model, we need a way to evaluate how well our model is performing. We can compare the model's predictions with the actual targets using the following method:

* Calculate the difference between the two matrices (`preds` and `targets`).
* Square all elements of the difference matrix to remove negative values.
* Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the **mean squared error** (MSE).

In [11]:
diff=preds-targets
diff

tensor([[157.2935, -68.6896],
        [198.5530, -89.7588],
        [223.4912, -94.8401],
        [181.9757, -79.8857],
        [168.7586, -79.3294]], grad_fn=<SubBackward0>)

In [12]:
torch.sum(diff*diff)

tensor(210151.9062, grad_fn=<SumBackward0>)

In [13]:
def MSE(t1,t2):
  diff=t1-t2
  return torch.sum(diff*diff)/diff.numel()

`torch.sum` returns the sum of all the elements in a tensor. The `.numel` method of a tensor returns the number of elements in a tensor. Let's compute the mean squared error for the current predictions of our model.

## Compute gradients

With PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t. to the weights and biases because they have `requires_grad` set to `True`. We'll see how this is useful in just a moment.

In [14]:
loss=MSE(preds,targets)
print(loss)

tensor(21015.1914, grad_fn=<DivBackward0>)


In [15]:
import math

In [16]:
math.sqrt(49691)

222.9147819234965

Here’s how we can interpret the result: *On average, each element in the prediction differs from the actual target by the square root of the loss*. And that’s pretty bad, considering the numbers we are trying to predict are themselves in the range 50–200. The result is called the *loss* because it indicates how bad the model is at predicting the target variables. It represents information loss in the model: the lower the loss, the better the model.

## Compute gradients

With PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t. to the weights and biases because they have `requires_grad` set to `True`. We'll see how this is useful in just a moment.

In [17]:
# Compute gradients
loss.backward()

In [18]:
print(w)

tensor([[ 1.0989,  1.5503,  0.6631],
        [-0.8992,  0.5920,  0.6517]], requires_grad=True)


In [19]:
print(w.grad)

tensor([[15840.0703, 16396.9883, 10195.9414],
        [-7011.1094, -7252.0513, -4541.5542]])


In [20]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


Before we proceed, we reset the gradients to zero by invoking the `.zero_()` method. We need to do this because PyTorch accumulates gradients. Otherwise, the next time we invoke `.backward` on the loss, the new gradient values are added to the existing gradients, which may lead to unexpected results.

## Train the model using gradient descent

As seen above, we reduce the loss and improve our model using the gradient descent optimization algorithm. Thus, we can _train_ the model using the following steps:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

Let's implement the above step by step.

In [21]:
#Generate prediction
preds=model(inputs)
print(preds)

tensor([[213.2935,   1.3104],
        [279.5530,  11.2412],
        [342.4912,  38.1599],
        [203.9757, -42.8857],
        [271.7586,  39.6706]], grad_fn=<AddBackward0>)


In [22]:
#Calculate loss
loss=MSE(preds,targets)
loss


tensor(21015.1914, grad_fn=<DivBackward0>)

In [23]:
loss.backward()

In [24]:
print(w.grad)
print(b.grad)

tensor([[15840.0703, 16396.9883, 10195.9414],
        [-7011.1094, -7252.0513, -4541.5542]])
tensor([186.0144, -82.5007])


In [25]:
w

tensor([[ 1.0989,  1.5503,  0.6631],
        [-0.8992,  0.5920,  0.6517]], requires_grad=True)

In [26]:
w.grad*1e-5

tensor([[ 0.1584,  0.1640,  0.1020],
        [-0.0701, -0.0725, -0.0454]])

In [27]:
#Adjust weights and reset gradients
with torch.no_grad():
  w-=w.grad*1e-5
  b-=b.grad *1e-5
  w.grad.zero_()
  b.grad.zero_()

with torch.no_grad() indicates pytorch that we shouldn't track,calculate or modify gradients while updating the weights and bises 

In [28]:
print(w)
print(b)

tensor([[ 0.9405,  1.3863,  0.5611],
        [-0.8291,  0.6645,  0.6971]], requires_grad=True)
tensor([ 0.6949, -0.7297], requires_grad=True)


In [29]:
#Calculate loss
preds=model(inputs)
loss=MSE(preds,targets)
print(loss)

tensor(14221.7363, grad_fn=<DivBackward0>)


## Train for multiple epochs

To reduce the loss further, we can repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an _epoch_. Let's train the model for 100 epochs.

In [30]:
# Train for 100 epochs
for i in range(100):
    preds = model(inputs)
    loss = MSE(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [31]:
preds=model(inputs)
loss=MSE(preds,targets)
print(loss)

tensor(62.3376, grad_fn=<DivBackward0>)


In [32]:
preds

tensor([[ 60.5224,  68.9648],
        [ 80.5956,  99.5580],
        [116.9625, 137.6465],
        [ 39.5378,  29.8874],
        [ 88.4819, 121.2196]], grad_fn=<AddBackward0>)

In [33]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

## Save and upload your notebook

Whether you're running this Jupyter notebook online or on your computer, it's essential to save your work from time to time. You can continue working on a saved notebook later or share it with friends and colleagues to let them execute your code. [Jovian](https://jovian.ai/platform-features) offers an easy way of saving and sharing your Jupyter notebooks online.

First, you need to install the Jovian python library if it isn't already installed.

In [34]:
!pip install jovian --upgrade -q

In [35]:
import jovian

In [36]:
jovian.commit(project='02-linear-regression')

[jovian] Detected Colab notebook...
[jovian] Please enter your API key ( from https://jovian.ai/ ):
API KEY: ··········
[jovian] Uploading colab notebook to Jovian...
Committed successfully! https://jovian.ai/faisal00001/02-linear-regression


'https://jovian.ai/faisal00001/02-linear-regression'

In [37]:
import torch.nn as nn

In [38]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [39]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

In [40]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

## Dataset and DataLoader

We'll create a `TensorDataset`, which allows access to rows from `inputs` and `targets` as tuples, and provides standard APIs for working with many different types of datasets in PyTorch.

In [41]:
from torch.utils.data import TensorDataset

In [42]:
train_ds=TensorDataset(inputs,targets)#Here train_ds is a tensor dataset object
train_ds[0:3]#To see first 3 values

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

The `TensorDataset` allows us to access a small section of the training data using the array indexing notation (`[0:3]` in the above code). It returns a tuple with two elements. The first element contains the input variables for the selected rows, and the second contains the targets.

We'll also create a `DataLoader`, which can split the data into batches of a predefined size while training. It also provides other utilities like shuffling and random sampling of the data.

##Use
We will use the tensor dataset to a dataloader.We will use 15 elements to 3 batches

In [43]:
from torch.utils.data import DataLoader

In [44]:
#Define the dataloader
batch_size=5
train_dl=DataLoader(train_ds,batch_size,shuffle=True)


##shuffle=True
When dataloader wants create batches so creating before batches we randomly shuffle data

##Example

In [45]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

In [46]:
for xb,yb in train_dl:#xb->Batch of inputs and yb batch of targets
  print(xb)
  print(yb)
  break
  #As batch size is 5 we see first 5.Here first five rows comes randomly for input and targets

tensor([[ 92.,  87.,  64.],
        [ 73.,  67.,  43.],
        [102.,  43.,  37.],
        [ 91.,  87.,  65.],
        [ 69.,  96.,  70.]])
tensor([[ 82., 100.],
        [ 56.,  70.],
        [ 22.,  37.],
        [ 80., 102.],
        [103., 119.]])


## nn.Linear

Instead of initializing the weights & biases manually, we can define the model using the `nn.Linear` class from PyTorch, which does it automatically.

In [47]:
#Define model
model=nn.Linear(3,2)#Here as we have 3 inputs so 3 and here we have two 2 out apple and orange
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.1388, -0.1438, -0.1077],
        [-0.5508,  0.5769, -0.0903]], requires_grad=True)
Parameter containing:
tensor([ 0.1595, -0.4718], requires_grad=True)


In [48]:
#Parameters
list(model.parameters())

[Parameter containing:
 tensor([[ 0.1388, -0.1438, -0.1077],
         [-0.5508,  0.5769, -0.0903]], requires_grad=True),
 Parameter containing:
 tensor([ 0.1595, -0.4718], requires_grad=True)]

In [49]:
preds=model(inputs)
print(preds)

tensor([[ -3.9738,  -5.9157],
        [ -6.7573,  -5.6132],
        [-13.2830,  23.6691],
        [  4.1512, -35.1932],
        [-11.6090,  10.5782],
        [ -3.6911,  -7.0434],
        [ -6.7212,  -6.2804],
        [-13.2519,  23.0279],
        [  3.8686, -34.0654],
        [-11.8556,  11.0387],
        [ -3.9377,  -6.5829],
        [ -6.4746,  -6.7409],
        [-13.3191,  24.3363],
        [  4.3978, -35.6537],
        [-11.8917,  11.7059]], grad_fn=<AddmmBackward>)


In [50]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

## Loss Function

Instead of defining a loss function manually, we can use the built-in loss function `mse_loss`.

In [51]:
#Import nn.functional
import torch.nn.functional as F

In [52]:
loss_fn=F.mse_loss

In [53]:
loss=loss_fn(model(inputs),targets)
print(loss)

tensor(8824.9912, grad_fn=<MseLossBackward>)


## Optimizer

Instead of manually manipulating the model's weights & biases using gradients, we can use the optimizer `optim.SGD`. SGD is short for "stochastic gradient descent". The term _stochastic_ indicates that samples are selected in random batches instead of as a single group.

In [54]:
#Define optimizer
opt=torch.optim.SGD(model.parameters(),lr=1e-5)

Note that `model.parameters()` is passed as an argument to `optim.SGD` so that the optimizer knows which matrices should be modified during the update step. Also, we can specify a learning rate that controls the amount by which the parameters are modified.

## Train the model

We are now ready to train the model. We'll follow the same process to implement gradient descent:

1. Generate predictions

2. Calculate the loss

3. Compute gradients w.r.t the weights and biases

4. Adjust the weights by subtracting a small quantity proportional to the gradient

5. Reset the gradients to zero

The only change is that we'll work batches of data instead of processing the entire training data in every iteration. Let's define a utility function `fit` that trains the model for a given number of epochs.

In [55]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Some things to note above:

* We use the data loader defined earlier to get batches of data for every iteration.

* Instead of updating parameters (weights and biases) manually, we use `opt.step` to perform the update and `opt.zero_grad` to reset the gradients to zero.

* We've also added a log statement that prints the loss from the last batch of data for every 10th epoch to track training progress. `loss.item` returns the actual value stored in the loss tensor.

Let's train the model for 100 epochs.

In [61]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 3.0273
Epoch [20/100], Loss: 2.4912
Epoch [30/100], Loss: 3.1003
Epoch [40/100], Loss: 4.3010
Epoch [50/100], Loss: 2.0109
Epoch [60/100], Loss: 2.5864
Epoch [70/100], Loss: 2.2721
Epoch [80/100], Loss: 1.1962
Epoch [90/100], Loss: 2.9765
Epoch [100/100], Loss: 1.4914


In [63]:
preds=model(inputs)
preds

tensor([[ 57.0600,  70.4539],
        [ 81.5116,  99.8162],
        [119.3451, 134.6076],
        [ 21.1244,  38.2598],
        [100.7969, 116.8384],
        [ 55.8006,  69.3614],
        [ 81.2685,  99.7621],
        [119.5834, 135.1332],
        [ 22.3838,  39.3523],
        [101.8132, 117.8769],
        [ 56.8169,  70.3999],
        [ 80.2522,  98.7237],
        [119.5882, 134.6617],
        [ 20.1081,  37.2214],
        [102.0564, 117.9309]], grad_fn=<AddmmBackward>)

In [64]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

Indeed, the predictions are quite close to our targets. We have a trained a reasonably good model to predict crop yields for apples and oranges by looking at the average temperature, rainfall, and humidity in a region. We can use it to make predictions of crop yields for new regions by passing a batch containing a single row of input.

In [66]:
model(torch.tensor([[75, 63, 44.]]))#Here we know the temperature ,rainfall ,humadity and we are now predicting the yield of Apple.Here inputs are pass to linear model and model predict the result

tensor([[53.4275, 67.3788]], grad_fn=<AddmmBackward>)

The predicted yield of apples is 54.3 tons per hectare, and that of oranges is 68.3 tons per hectare.

## Machine Learning vs. Classical Programming

The approach we've taken in this tutorial is very different from programming as you might know it. Usually, we write programs that take some inputs, perform some operations, and return a result. 

However, in this notebook, we've defined a "model" that assumes a specific relationship between the inputs and the outputs, expressed using some unknown parameters (weights & biases). We then show the model some know inputs and outputs and _train_ the model to come up with good values for the unknown parameters. Once trained, the model can be used to compute the outputs for new inputs.

This paradigm of programming is known as _machine learning_, where we use data to figure out the relationship between inputs and outputs. _Deep learning_ is a branch of machine learning that uses matrix operations, non-linear activation functions and gradient descent to build and train models. Andrej Karpathy, the director of AI at Tesla Motors, has written a great blog post on this topics, titled [Software 2.0](https://medium.com/@karpathy/software-2-0-a64152b37c35).

This picture from book [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python) by Francois Chollet captures the difference between classical programming and machine learning:

![](https://i.imgur.com/oJEQe7k.png)





## Commit and update the notebook

As a final step, we can record a new version of the notebook using the `jovian` library.

In [67]:
import jovian

In [68]:
jovian.commit(project='02-linear-regression')

[jovian] Detected Colab notebook...
[jovian] Uploading colab notebook to Jovian...
Committed successfully! https://jovian.ai/faisal00001/02-linear-regression


'https://jovian.ai/faisal00001/02-linear-regression'